<h4>Building a service server</h4>

This is an exact replication of creating an service ([here](https://docs.ros.org/en/galactic/Tutorials/Beginner-Client-Libraries/Writing-A-Simple-Py-Service-And-Client.html))  The first step is to define it by creating the requisite paths, generate all the folders, and create a package.

In [6]:
import os
import subprocess

home = os.path.expanduser('~')
workspace = home +'/ros2_ws/'
sourceFiles = workspace + 'src/'
packageName = 'py_srvcli'

os.chdir(home)

if not os.path.exists(sourceFiles):
    os.makedirs(sourceFiles)
os.chdir(sourceFiles)
if not os.path.exists(packageName):
    reply = subprocess.run(['ros2','pkg','create','--build-type','ament_python',
                            packageName,'--dependencies','rclpy',
           'example_interfaces'],capture_output=True) #same as !ros2 pkg create --build-type ament_python py_srvcli --dependencies rclpy example_interfaces
    

In [7]:
reply

CompletedProcess(args=['ros2', 'pkg', 'create', '--build-type', 'ament_python', 'py_srvcli', '--dependencies', 'rclpy', 'example_interfaces'], returncode=0, stdout=b"going to create a new package\npackage name: py_srvcli\ndestination directory: /home/parallels/ros2_ws/src\npackage format: 3\nversion: 0.0.0\ndescription: TODO: Package description\nmaintainer: ['parallels <parallels@todo.todo>']\nlicenses: ['TODO: License declaration']\nbuild type: ament_python\ndependencies: ['rclpy', 'example_interfaces']\ncreating folder ./py_srvcli\ncreating ./py_srvcli/package.xml\ncreating source folder\ncreating folder ./py_srvcli/py_srvcli\ncreating ./py_srvcli/setup.py\ncreating ./py_srvcli/setup.cfg\ncreating folder ./py_srvcli/resource\ncreating ./py_srvcli/resource/py_srvcli\ncreating ./py_srvcli/py_srvcli/__init__.py\ncreating folder ./py_srvcli/test\ncreating ./py_srvcli/test/test_copyright.py\ncreating ./py_srvcli/test/test_flake8.py\ncreating ./py_srvcli/test/test_pep257.py\n", stderr=b''

Next we set up the service definition file.  This defines the data types for *Requests* and  *Response*, separated by a dashed line.  In this case, the request has two parameters, a and b, both 64 bit integers, the result (only one parameter) is also a 64 bit integers.  First change the directory to the service folder and then save it.

In [2]:
os.chdir(sourceFiles + packageName)

In [3]:
%%writefile add.srv

int64 a
int64 b
---
int64 sum

Writing Fibonacci.action


We then should update the package xml, adding your name and such.
```xml
<description>Python client server tutorial</description>
<maintainer email="you@email.com">Your Name</maintainer>
<license>Apache License 2.0</license>
```

In [4]:
os.chdir(sourceFiles + packageName)

In [8]:
%%writefile package.xml
<?xml version="1.0"?>
<?xml-model href="http://download.ros.org/schema/package_format3.xsd" schematypens="http://www.w3.org/2001/XMLSchema"?>
<package format="3">
  <name>py_srvcli</name>
  <version>0.0.0</version>
  <description>TODO: Package description</description>
  <maintainer email="parallels@todo.todo">parallels</maintainer>
  <license>TODO: License declaration</license>

  <depend>rclpy</depend>
  <depend>example_interfaces</depend>

  <test_depend>ament_copyright</test_depend>
  <test_depend>ament_flake8</test_depend>
  <test_depend>ament_pep257</test_depend>
  <test_depend>python3-pytest</test_depend>

  <export>
    <build_type>ament_python</build_type>
  </export>
</package>

Writing package.xml


Likewise, you shoudl update the setup.py file
```python
maintainer='Your Name',
maintainer_email='you@email.com',
description='Python client server tutorial',
license='Apache License 2.0',
```

In [6]:
os.chdir(sourceFiles + packageName)

In [9]:
%%writefile setup.py
from setuptools import setup

package_name = 'py_srvcli'

setup(
    name=package_name,
    version='0.0.0',
    packages=[package_name],
    data_files=[
        ('share/ament_index/resource_index/packages',
            ['resource/' + package_name]),
        ('share/' + package_name, ['package.xml']),
    ],
    install_requires=['setuptools'],
    zip_safe=True,
    maintainer='parallels',
    maintainer_email='parallels@todo.todo',
    description='TODO: Package description',
    license='TODO: License declaration',
    tests_require=['pytest'],
    entry_points={
        'console_scripts': [
        ],
    },
)

Writing setup.py


os.chdir(home + '/ros2_ws')

In [10]:
os.chdir(home + '/ros2_ws')
!rosdep install -i --from-path src --rosdistro galactic -y


ERROR: your rosdep installation has not been initialized yet.  Please run:

    sudo rosdep init
    rosdep update



rosdep install -i --from-path src --rosdistro galactic -yLastly we have to build it by running *colcon build* in the workspace directory

In [13]:
import subprocess
os.chdir(home + '/ros2_ws')
reply = subprocess.run(['colcon','build','--packages-select','py_srvcli'],capture_output=True)

In [14]:
reply

CompletedProcess(args=['colcon', 'build', '--packages-select', 'py_srvcli'], returncode=1, stdout=b'Starting >>> py_srvcli\n\nSummary: 0 packages finished [0.68s]\n  1 package failed: py_srvcli\n  1 package had stderr output: py_srvcli\n', stderr=b"--- stderr: py_srvcli\npackage init file 'py_srvcli/__init__.py' not found (or not a regular file)\nerror: can't copy 'resource/py_srvcli': doesn't exist or not a regular file\n---\nFailed   <<< py_srvcli [0.57s, exited with code 1]\n")

In [15]:
!. install/setup.bash

In [16]:
!ros2 interface show action_tutorials_interfaces/action/Fibonacci

Unknown package 'action_tutorials_interfaces'


In [17]:
!export ROS_DOMAIN_ID=0

Now you have to go to the terminal and run 
```bash
cd ~/ros2_ws
export ROS_DOMAIN_ID=0
. install/setup.bash
ros2 interface show action_tutorials_interfaces/action/Fibonacci
```
This will test the action definition

#### Server code

the next step is to write the python code that will define the server.  We first define the server, importing the message type we defined above and defining a callback that returns the result.  

In [27]:
%%writefile ~/ros2_ws/fibonacci_action_server.py

import rclpy
from rclpy.action import ActionServer
from rclpy.node import Node

from action_tutorials_interfaces.action import Fibonacci

class FibonacciActionServer(Node):

    def __init__(self):
        super().__init__('fibonacci_action_server')
        self._action_server = ActionServer(
            self,Fibonacci,'fibonacci',self.execute_callback)
        print('set up and waiting')

    def execute_callback(self, goal_handle):
        self.get_logger().info('Executing goal...')

        sequence = [0, 1]

        for i in range(1, goal_handle.request.order):
            sequence.append(sequence[i] + sequence[i-1])

        goal_handle.succeed()

        result = Fibonacci.Result()
        result.sequence = sequence
        return result

def main(args=None):
    rclpy.init(args=args)

    fibonacci_action_server = FibonacciActionServer()

    rclpy.spin(fibonacci_action_server)


if __name__ == '__main__':
    main()

Writing /home/parallels/ros2_ws/fibonacci_action_server.py


Now you can run it in the terminal by typing:
```bash
cd ~/ros2_ws
python3 fibonacci_action_server.py
```
Then open up a second terminal, and type:
```bash
cd ~/ros2_ws
. install/setup.bash
ros2 action send_goal fibonacci action_tutorials_interfaces/action/Fibonacci "{order: 5}"
```

#### Client code

Our next step is to call the server from whithin python and get back the result.  So (similar to all the iRobot calls) we set up a callback on sending out the goal and then on getting the result.

In [ ]:
%%writefile ~/ros2_ws/fibonacci_action_client.py

import rclpy
from rclpy.action import ActionClient
from rclpy.node import Node

from action_tutorials_interfaces.action import Fibonacci


class FibonacciActionClient(Node):

    def __init__(self):
        super().__init__('fibonacci_action_client')
        self._action_client = ActionClient(self, Fibonacci, 'fibonacci')

    def send_goal(self, order):
        print('sending goal')
        goal_msg = Fibonacci.Goal()
        goal_msg.order = order

        self._action_client.wait_for_server()
        print('done waiting')
        self._send_goal_future = self._action_client.send_goal_async(goal_msg)

        self._send_goal_future.add_done_callback(self.goal_response_callback)

    def goal_response_callback(self, future):
        
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().info('Goal rejected :(')
            return

        self.get_logger().info('Goal accepted :)')

        self._get_result_future = goal_handle.get_result_async()
        self._get_result_future.add_done_callback(self.get_result_callback)

    def get_result_callback(self, future):
        result = future.result().result
        self.get_logger().info('Result: {0}'.format(result.sequence))
        rclpy.shutdown()


def main(args=None):
    rclpy.init(args=args)

    action_client = FibonacciActionClient()
    print('set up')
    action_client.send_goal(10)

    rclpy.spin(action_client)


if __name__ == '__main__':
    main()

### More advanced server

We can also have a server that returns values as it is running - leveraging hte *partial_sequence* feedback parameter.


In [ ]:
%%writefile ~/ros2_ws/fibonacci_action_server2.py

import time

import rclpy
from rclpy.action import ActionServer
from rclpy.node import Node

from action_tutorials_interfaces.action import Fibonacci


class FibonacciActionServer(Node):

    def __init__(self):
        super().__init__('fibonacci_action_server')
        self._action_server = ActionServer(
            self,
            Fibonacci,
            'fibonacci',
            self.execute_callback)

    def execute_callback(self, goal_handle):
        self.get_logger().info('Executing goal...')

        feedback_msg = Fibonacci.Feedback()
        feedback_msg.partial_sequence = [0, 1]

        for i in range(1, goal_handle.request.order):
            feedback_msg.partial_sequence.append(
                feedback_msg.partial_sequence[i] + feedback_msg.partial_sequence[i-1])
            self.get_logger().info('Feedback: {0}'.format(feedback_msg.partial_sequence))
            goal_handle.publish_feedback(feedback_msg)
            time.sleep(1)

        goal_handle.succeed()

        result = Fibonacci.Result()
        result.sequence = feedback_msg.partial_sequence
        return result


def main(args=None):
    rclpy.init(args=args)

    fibonacci_action_server = FibonacciActionServer()

    rclpy.spin(fibonacci_action_server)


if __name__ == '__main__':
    main()

And then your client code looks like this.

In [ ]:
i%%writefile ~/ros2_ws/fibonacci_action_client2.py

mport rclpy
from rclpy.action import ActionClient
from rclpy.node import Node

from action_tutorials_interfaces.action import Fibonacci


class FibonacciActionClient(Node):

    def __init__(self):
        super().__init__('fibonacci_action_client')
        self._action_client = ActionClient(self, Fibonacci, 'fibonacci')

    def send_goal(self, order):
        goal_msg = Fibonacci.Goal()
        goal_msg.order = order

        self._action_client.wait_for_server()

        self._send_goal_future = self._action_client.send_goal_async(goal_msg, feedback_callback=self.feedback_callback)

        self._send_goal_future.add_done_callback(self.goal_response_callback)

    def goal_response_callback(self, future):
        goal_handle = future.result()
        if not goal_handle.accepted:
            self.get_logger().info('Goal rejected :(')
            return

        self.get_logger().info('Goal accepted :)')

        self._get_result_future = goal_handle.get_result_async()
        self._get_result_future.add_done_callback(self.get_result_callback)

    def get_result_callback(self, future):
        result = future.result().result
        self.get_logger().info('Result: {0}'.format(result.sequence))
        rclpy.shutdown()

    def feedback_callback(self, feedback_msg):
        feedback = feedback_msg.feedback
        self.get_logger().info('Received feedback: {0}'.format(feedback.partial_sequence))


def main(args=None):
    rclpy.init(args=args)

    action_client = FibonacciActionClient()

    action_client.send_goal(10)

    rclpy.spin(action_client)


if __name__ == '__main__':
    main()